# IF WANT TO USE THIS CODE FURTHER, PLEASE CHANGE THE DIRECTORY + CREATE THE CLASSES.TXT MANUALLY

## This code is to create a structure as similiar as yolo

Create copy if want to implement further, and change the paths

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Create the Directory Structure:

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
drive_path = '/content/drive/MyDrive/Object Detection/data'
labels_source = os.path.join(drive_path, 'new_data')
images_source = os.path.join(drive_path, 'new_data/frames')
dataset_path = '/content/drive/MyDrive/Object Detection/data/dataset-new'  # New structured dataset location


Conversi XML into TXT

In [ ]:
import os
import xml.etree.ElementTree as ET
from pathlib import Path

def convert_voc_to_yolo(xml_path, output_dir, classes):
    """
    Convert a single VOC XML file to YOLO format
    """
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Get image dimensions
        size = root.find('size')
        if size is None:
            print(f"Warning: No size element found in {xml_path}")
            return False

        width_elem = size.find('width')
        height_elem = size.find('height')

        if width_elem is None or height_elem is None:
            print(f"Warning: Missing width or height in {xml_path}")
            return False

        width = int(width_elem.text)
        height = int(height_elem.text)

        # Prepare output file
        txt_filename = Path(xml_path).stem + '.txt'
        txt_path = os.path.join(output_dir, txt_filename)

        objects_found = 0
        with open(txt_path, 'w') as f:
            for obj in root.findall('object'):
                # Get class name
                name_elem = obj.find('name')
                if name_elem is None:
                    print(f"Warning: Object without name in {xml_path}")
                    continue

                class_name = name_elem.text

                # Check if class is in our list
                if class_name not in classes:
                    print(f"Warning: Class '{class_name}' not in classes list, skipping...")
                    continue

                class_id = classes.index(class_name)

                # Get bounding box coordinates
                bndbox = obj.find('bndbox')
                if bndbox is None:
                    print(f"Warning: Object without bndbox in {xml_path}")
                    continue

                xmin_elem = bndbox.find('xmin')
                ymin_elem = bndbox.find('ymin')
                xmax_elem = bndbox.find('xmax')
                ymax_elem = bndbox.find('ymax')

                if None in [xmin_elem, ymin_elem, xmax_elem, ymax_elem]:
                    print(f"Warning: Incomplete bndbox in {xml_path}")
                    continue

                xmin = int(xmin_elem.text)
                ymin = int(ymin_elem.text)
                xmax = int(xmax_elem.text)
                ymax = int(ymax_elem.text)

                # Convert to YOLO format
                x_center = (xmin + xmax) / 2 / width
                y_center = (ymin + ymax) / 2 / height
                w = (xmax - xmin) / width
                h = (ymax - ymin) / height

                # Ensure values are within [0,1] range
                x_center = max(0, min(1, x_center))
                y_center = max(0, min(1, y_center))
                w = max(0, min(1, w))
                h = max(0, min(1, h))

                # Write to file
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")
                objects_found += 1

        print(f"Converted {xml_path.name}: {objects_found} objects")
        return objects_found > 0

    except Exception as e:
        print(f"Error processing {xml_path}: {str(e)}")
        return False

def convert_dataset(voc_annotations_dir, output_dir, classes_file):
    """
    Convert all VOC XML files in a directory to YOLO format
    """
    # Check if directories exist
    if not os.path.exists(voc_annotations_dir):
        print(f"Error: VOC annotations directory '{voc_annotations_dir}' does not exist")
        return

    if not os.path.exists(classes_file):
        print(f"Error: Classes file '{classes_file}' does not exist")
        return

    # Read class names
    with open(classes_file, 'r') as f:
        classes = [line.strip() for line in f.readlines() if line.strip()]

    print(f"Classes loaded: {classes}")

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Find XML files
    xml_files = list(Path(voc_annotations_dir).glob('*.xml'))
    print(f"Found {len(xml_files)} XML files")

    if len(xml_files) == 0:
        print("No XML files found. Check the directory path.")
        return

    # Convert each XML file
    successful_conversions = 0
    for xml_file in xml_files:
        if convert_voc_to_yolo(xml_file, output_dir, classes):
            successful_conversions += 1

    print(f"Conversion completed: {successful_conversions}/{len(xml_files)} files converted successfully")

if __name__ == "__main__":
    # Your paths
    convert_dataset(
        voc_annotations_dir=labels_source,
        output_dir="/content/drive/MyDrive/Object Detection/data/newYoloLabel",
        classes_file="/content/drive/MyDrive/Object Detection/data/newYoloLabel/classes.txt"
    )

Classes loaded: ['person', 'washing_machine', 'mineral_water', 'presser', 'phone']
Found 149 XML files
Converted frame_20250917_0950380001.xml: 9 objects
Converted frame_20250917_0950380017.xml: 2 objects
Converted frame_20250917_0950380028.xml: 2 objects
Converted frame_20250917_0950380030.xml: 2 objects
Converted frame_20250917_0950380016.xml: 2 objects
Converted frame_20250917_0950380005.xml: 9 objects
Converted frame_20250917_0950380031.xml: 2 objects
Converted frame_20250917_0950380024.xml: 2 objects
Converted frame_20250917_0950380007.xml: 9 objects
Converted frame_20250917_0950380010.xml: 9 objects
Converted frame_20250917_0950380019.xml: 2 objects
Converted frame_20250917_0950380020.xml: 2 objects
Converted frame_20250917_0950380022.xml: 2 objects
Converted frame_20250917_0950380013.xml: 3 objects
Converted frame_20250917_0950380014.xml: 2 objects
Converted frame_20250917_0950380032.xml: 2 objects
Converted frame_20250917_0950380027.xml: 2 objects
Converted frame_20250917_09503

Verify the classes.txt

In [ ]:
# Run this separately to check your classes file
classes_file = "/content/drive/MyDrive/Object Detection/data/newYoloLabel/classes.txt"
with open(classes_file, 'r') as f:
    classes = [line.strip() for line in f.readlines() if line.strip()]
print("Classes in file:", classes)

Classes in file: ['person', 'washing_machine', 'mineral_water', 'presser', 'phone']


Check xml files

In [ ]:
# Run this to check the structure of one XML file
sample_xml = list(Path("/content/drive/MyDrive/Object Detection/data/new_data/").glob('*.xml'))[0]
print("Sample XML content:")
with open(sample_xml, 'r') as f:
    print(f.read())

Sample XML content:
<annotation>
	<folder>my-project-name</folder>
	<filename>frame_20250917_0950380001.jpg</filename>
	<path>/my-project-name/frame_20250917_0950380001.jpg</path>
	<source>
		<database>Unspecified</database>
	</source>
	<size>
		<width>1024</width>
		<height>576</height>
		<depth>3</depth>
	</size>
	<object>
		<name>person</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>909</xmin>
			<ymin>92</ymin>
			<xmax>943</xmax>
			<ymax>163</ymax>
		</bndbox>
	</object>
	<object>
		<name>person</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>886</xmin>
			<ymin>87</ymin>
			<xmax>922</xmax>
			<ymax>144</ymax>
		</bndbox>
	</object>
	<object>
		<name>washing_machine</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>556</xmin>
			<ymin>192</ymin>
			<xmax>670</xmax>
			<ymax>350</ymax>
		</bndbox>
	</obj

check if files existed

In [ ]:
# Verify the paths
import os
print("VOC annotations dir exists:", os.path.exists("/content/drive/MyDrive/Object Detection/data/new_data"))
print("Classes file exists:", os.path.exists("/content/drive/MyDrive/Object Detection/data/newYoloLabel/classes.txt"))

VOC annotations dir exists: True
Classes file exists: True
